In [1]:
import json
import sys
from functools import partial

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim.lr_scheduler as lr_scheduler
import wandb
from ignite.contrib.handlers import wandb_logger
from ignite.engine import (Engine, Events, create_supervised_evaluator,
                           create_supervised_trainer)
from ignite.handlers import ModelCheckpoint
from ignite.handlers.param_scheduler import LRScheduler
from ignite.metrics import Accuracy, Loss
from scipy.io.arff import loadarff
from sklearn.model_selection import train_test_split
from torch import nn
from torch.functional import F
from torch.utils.data import DataLoader, Dataset, SubsetRandomSampler

sys.path.append('../')
from src.datasets import make_dataset
from src.models import TransformerClassification
from src.utils import build_optimizer, str2torch

In [2]:
with open('../configs/transformer_87.json') as f:
    config = json.load(f)

In [3]:
train_dataloader, val_dataloader, test_dataloader = make_dataset(config)

In [4]:
next(iter(train_dataloader))[0].shape

torch.Size([256, 150])

In [5]:
# Initialize your model
wandb.init(entity='ts-robustness', project='ml-course', config=config, tags=['hypersearch'])

device = 'cuda' if torch.cuda.is_available() else 'cpu'
config['train']['optimizer'] = str2torch(config['train']['optimizer'])

model = TransformerClassification(config).to(device)

# Initialize your optimizer and criterion
optimizer = build_optimizer(config, model)
criterion = nn.CrossEntropyLoss()

def train_step(engine, batch):
    model.train()
    optimizer.zero_grad()
    x, y = batch[0].to(device), batch[1].to(device)
    y_pred = model(x)
    loss = criterion(y_pred, y.long())
    loss.backward()
    optimizer.step()
    return loss.item()

trainer = Engine(train_step)

def validation_step(engine, batch):
    model.eval()
    with torch.no_grad():
        x, y = batch[0].to(device), batch[1].to(device)
        y_pred = model(x)
        return y_pred, y

train_evaluator = Engine(validation_step)
val_evaluator = Engine(validation_step)
test_evaluator = Engine(validation_step)

# Attach metrics to the evaluators
metrics = {
    'accuracy': Accuracy(output_transform=lambda x: (torch.argmax(x[0], dim=1), x[1])),
    'loss': Loss(criterion, output_transform=lambda x: (x[0], x[1].long()))
}

for name, metric in metrics.items():
    metric.attach(train_evaluator, name)
    metric.attach(val_evaluator, name)
    metric.attach(test_evaluator, name)


checkpoint_handler = ModelCheckpoint(dirname=wandb.run.dir + '/../saved_models', filename_prefix='best',
                                     n_saved=1, require_empty=False,
                                     score_function=lambda engine: engine.state.metrics['accuracy'],
                                     score_name="accuracy", global_step_transform=lambda *_: trainer.state.epoch)
test_evaluator.add_event_handler(Events.EPOCH_COMPLETED, checkpoint_handler, {"model": model})


@trainer.on(Events.EPOCH_COMPLETED)
def log_training_results(trainer):
    train_evaluator.run(train_dataloader)
    metrics = train_evaluator.state.metrics
    print("Training Results - Epoch: {}  Avg accuracy: {:.2f} Avg loss: {:.4f}"
          .format(trainer.state.epoch, metrics['accuracy'], metrics['loss']))
    wandb.log({"train_accuracy": metrics['accuracy'],
               "train_loss": metrics['loss']})

@trainer.on(Events.EPOCH_COMPLETED)
def log_validation_results(trainer):
    val_evaluator.run(val_dataloader)
    metrics = val_evaluator.state.metrics
    print("Validation Results - Epoch: {}  Avg accuracy: {:.2f} Avg loss: {:.4f}"
          .format(trainer.state.epoch, metrics['accuracy'], metrics['loss']))
    wandb.log({"val_accuracy": metrics['accuracy'],
               "val_loss": metrics['loss']})
    
@trainer.on(Events.EPOCH_COMPLETED)
def log_test_results(trainer):
    test_evaluator.run(test_dataloader)
    metrics = test_evaluator.state.metrics
    print("Test Results - Epoch: {}  Avg accuracy: {:.2f} Avg loss: {:.4f}"
          .format(trainer.state.epoch, metrics['accuracy'], metrics['loss']))
    wandb.log({"test_accuracy": metrics['accuracy'],
               "test_loss": metrics['loss']})


# Run the training loop
trainer.run(train_dataloader, max_epochs=config['train']['n_epoch'])
# wandb.finish()

wandb: Currently logged in as: gamma_function (ts-robustness). Use `wandb login --relogin` to force relogin


C:\Users\User\anaconda3\Lib\site-packages\torch\nn\functional.py:5476: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = scaled_dot_product_attention(q, k, v, attn_mask, dropout_p, is_causal)


Training Results - Epoch: 1  Avg accuracy: 0.57 Avg loss: 0.6739
Validation Results - Epoch: 1  Avg accuracy: 0.57 Avg loss: 0.6741
Test Results - Epoch: 1  Avg accuracy: 0.56 Avg loss: 0.6751
Training Results - Epoch: 2  Avg accuracy: 0.60 Avg loss: 0.6541
Validation Results - Epoch: 2  Avg accuracy: 0.60 Avg loss: 0.6547
Test Results - Epoch: 2  Avg accuracy: 0.59 Avg loss: 0.6594
Training Results - Epoch: 3  Avg accuracy: 0.67 Avg loss: 0.6230
Validation Results - Epoch: 3  Avg accuracy: 0.67 Avg loss: 0.6213
Test Results - Epoch: 3  Avg accuracy: 0.64 Avg loss: 0.6336
Training Results - Epoch: 4  Avg accuracy: 0.70 Avg loss: 0.5788
Validation Results - Epoch: 4  Avg accuracy: 0.70 Avg loss: 0.5776
Test Results - Epoch: 4  Avg accuracy: 0.69 Avg loss: 0.5882
Training Results - Epoch: 5  Avg accuracy: 0.75 Avg loss: 0.5098
Validation Results - Epoch: 5  Avg accuracy: 0.75 Avg loss: 0.5083
Test Results - Epoch: 5  Avg accuracy: 0.75 Avg loss: 0.5125
Training Results - Epoch: 6  Avg ac

State:
	iteration: 3390
	epoch: 30
	epoch_length: 113
	max_epochs: 30
	output: 0.3266116678714752
	batch: <class 'list'>
	metrics: <class 'dict'>
	dataloader: <class 'torch.utils.data.dataloader.DataLoader'>
	seed: <class 'NoneType'>
	times: <class 'dict'>

In [7]:
# Count the number of parameters in the transformer_encoder layer
transformer_encoder_params = sum(p.numel() for p in model.transformer_encoder.parameters())

# Count the number of parameters in the fc layer
fc_params = sum(p.numel() for p in model.fc.parameters())

print("Number of parameters in transformer_encoder:", transformer_encoder_params)
print("Number of parameters in fc layer:", fc_params)


Number of parameters in transformer_encoder: 214346
Number of parameters in fc layer: 187652


In [6]:
def req_grad(model: nn.Module, state: bool = True) -> None:
    """Set requires_grad of all model parameters to the desired value.

    :param model: the model
    :param state: desired value for requires_grad
    """
    for param in model.parameters():
        param.requires_grad_(state)

In [7]:
def BMI_attack(num_iterations):
    epsilon = 0.01
    req_grad(model, state=True)
    model.train()
    loss_function = torch.nn.BCELoss()
    
    correct = 0
    total = 0
    for input, target in test_dataloader:
        input = input.to(device)
        target = target.to(device)
        
        adversarial_input = input.clone().requires_grad_(True)  
        
        
        for _ in range(num_iterations):
            adversarial_input.requires_grad = True
            #print(adversarial_input.requires_grad)
            predictions = model(adversarial_input)
            loss = loss_function(predictions.flatten(), target)
            
            grad_ = torch.autograd.grad(loss, adversarial_input, retain_graph=True)[0]
            adversarial_input = adversarial_input.data + epsilon * torch.sign(grad_)
    
        adversarial_predictions = model(adversarial_input)
        correct += (adversarial_predictions.to('cpu').round().reshape(1,-1) == target.to('cpu')).sum().item()
        total += len(target)
    return correct/total, input, adversarial_input

In [8]:
acc, inp, adv_inp = BMI_attack(2)

ValueError: Using a target size (torch.Size([256])) that is different to the input size (torch.Size([512])) is deprecated. Please ensure they have the same size.